In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline
from scipy.optimize import minimize
from lsst_efd_client import EfdClient
from lsst.summit.utils import getCurrentDayObs_int, dayObsIntToString
from lsst.summit.utils.tmaUtils import TMAEventMaker, plotEvent, getAzimuthElevationDataForEvent
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import getEfdData

In [ ]:
client = EfdClient("usdf_efd")
eventMaker = TMAEventMaker()

In [ ]:
dayObs=20240222
seqNum = 1089
track = eventMaker.getEvent(dayObs, seqNum)


In [ ]:

def filterBadValues(values, maxDelta=0.1):
    """Filter out bad values from a dataset, replacing them in-place.                                                                       
                                                                                                                                            
    This function replaces non-physical points in the dataset with an                                                                       
    extrapolation of the preceding two values. No more than 3 successive data                                                               
    points are allowed to be replaced. Minimum length of the input is 3 points.                                                             
                                                                                                                                            
    Parameters                                                                                                                              
    ----------                                                                                                                              
    values : `list` or `np.ndarray`                                                                                                         
        The dataset containing the values to be filtered.                                                                                   
    maxDelta : `float`, optional                                                                                                            
        The maximum allowed difference between consecutive values. Values with                                                              
        a difference greater than `maxDelta` will be considered as bad values                                                               
        and replaced with an extrapolation.                                                                                                 
                                                                                                                                            
    Returns                                                                                                                                 
    -------                                                                                                                                 
    nBadPoints : `int`      
    """
    # Find non-physical points and replace with extrapolation. No more than 3                                                               
    # successive data points can be replaced.                                                                                               
    badCounter = 0
    consecutiveCounter = 0

    log = logging.getLogger(__name__)

    median = np.nanmedian(values)
    # if either of the the first two points are more than maxDelta away from                                                                
    # the median, replace them with the median                                                                                              
    for i in range(2):
        if abs(values[i] - median) > maxDelta:
            values[i] = median
            badCounter += 1
            log.warning(f"Replacing bad value at index {i} with median value")

    # from the second element of the array, walk through and calculate the                                                                  
    # difference between each element and the previous one. If the difference                                                               
    # is greater than maxDelta, replace the element with the average of the                                                                 
    # previous two elements from the point where we started replacing.                                                                      
    replacementValue = (values[1] + values[0]) / 2.0  # in case we have to replace the first value                                          
    for i in range(2, len(values)):
        if abs(values[i] - values[i-1]) > maxDelta:
            if consecutiveCounter < 3:
                consecutiveCounter += 1
                badCounter += 1
                log.warning(f"Replacing value at index {i} with {replacementValue}")
                values[i] = replacementValue
            else:
                log.warning(f"More than 3 consecutive replacements at index {i}. Stopping replacements"
                            " until the next good value.")
        else:
            consecutiveCounter = 0
            replacementValue = (values[1] + values[0]) / 2.0
    return badCounter


In [ ]:
azimuthData, elevationData = getAzimuthElevationDataForEvent(
        client, track, prePadding=-1.0, postPadding=-1.0)
thisEl = elevationData['actualPosition'].iloc[0]
azError = azimuthData['azError'].values
elError = elevationData['elError'].values
azError = azimuthData['azError'].values
elError = elevationData['elError'].values
azBadValues = filterBadValues(azError)
elBadValues = filterBadValues(elError)
azimuthData['azError'] = azError
elevationData['elError'] = elError
azError = azimuthData['azError']
elError = elevationData['elError']


In [ ]:
%matplotlib inline
elError.plot()
plt.show()

In [ ]:
fft = np.fft.fft(elError.values)

In [ ]:
for i in range(1,10):
    print(azimuthData['timestamp'][i] - azimuthData['timestamp'][i-1])

In [ ]:
fftFreq = np.fft.fftfreq(len(elError.values), d=0.1)


In [ ]:
plt.title(f"FFT of TMA Elevation oscillations - {dayObs} seqNum {seqNum}")
plt.plot(fftFreq, np.abs(fft))
plt.xlim(0,5)
plt.xlabel("Frequency (Hz)")
plt.savefig(f"plots/Elevation_Oscillations_{dayObs}_{seqNum}.png")